<a href="https://colab.research.google.com/github/micha-blip/Simple-article-reference-checker/blob/main/Reference_checker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Simple reference checker

This notebook provides tools to check the validity of references in scientific articles using DOIs. The goal is to help identify potentially fabricated or hallucinated references, which can be a concern with AI-generated content, and thereby contribute to improving the reliability and integrity of scientific literature.

In [128]:
import requests
import numpy as np
import pandas as pd
from Bio import Entrez # Import Entrez

def get_references_from_doi(doi, return_dataframe = True, verbose = True):
  """
  Fetches references for a given DOI, checks their existence via Crossref,
  and returns a summary or a DataFrame of the results.

  Args:
    doi (str): The DOI of the article to fetch references from.
    return_dataframe (bool, optional): If True, returns a pandas DataFrame
                                        with DOI and status. If False, returns
                                        a dictionary summary. Defaults to True.
    verbose (bool, optional): If True, prints detailed status during the
                              reference check. Defaults to True.

  Returns:
    pandas.DataFrame or dict: A DataFrame with reference DOIs and their status
                              ('article found', 'no article', 'no DOI'),
                              or a dictionary summarizing the counts of each status.
                              Returns None if the initial article DOI is not found.
  """

  api_url = f"https://api.crossref.org/v1/works/{doi}"
  headers = {"User-Agent": "SimpleFetcher"}
  print('Sending request...')

  try:
      response = requests.get(api_url, headers=headers)
      response.raise_for_status()  # Check for bad responses
      data = response.json()

      # Get the list of references from the response
      references = data["message"].get("reference", [])
      title = data["message"].get('title', [])
      title = str(title[0]).replace("\\n", " ").replace('<sup>+</sup>', "")
      print('Processing response for: ' + '\x1B[3m'  + title)

      doi_list = []

      if references:
          for i, ref in enumerate(references):
              ref_key = ref.get('key', f"substitute_key_{i}") # Use a substitute key if 'key' is missing
              if "DOI" in ref:
                  doi_list.append(ref['DOI'])
              elif "unstructured" in ref:
                  doi_list.append('not found')
              else:
                  doi_list.append('not found')

  except requests.exceptions.RequestException as e:
      print(f"An error occurred: The article was not found")
      return

  check_status = check_references_for_doi(doi_list, verbose)
  print("Done, found " + str(sum(check_status == "article found")) + " existing documents, " + str(sum(check_status == "no article")) + " non-existing documents" + " and " + str(sum(check_status == 'no DOI')) + " missing DOI")

  if verbose == False:
    result = {"Found" : int(sum(check_status == "article found")), "Not found" : int(sum(check_status == "no article")), "Missing DOI" : int(sum(check_status == 'no DOI'))}
    return pd.DataFrame(result, index = range(1))



  if return_dataframe == True:
    references_checked = pd.DataFrame(doi_list,  columns=['DOI'])
    references_checked['status'] = check_status
    return references_checked




def check_references_for_doi(doi_list, verbose):
  """
  Checks the existence of a list of DOIs using the PubMed API (Entrez).

  Args:
    doi_list (list): A list of DOIs (strings) to check. 'not found' indicates
                     a missing DOI in the original reference list.
    verbose (bool): If True, prints the status and title (if found) for each DOI.

  Returns:
    numpy.ndarray: A numpy array of strings indicating the status for each DOI
                   in the input list ('article found', 'no article', 'no DOI').
  """
  # Always tell Entrez who you are
  Entrez.email = "your_email@example.com" # Replace with your email address

  status = np.full(len(doi_list), "not processed")

  for i, doi in enumerate(doi_list):
    if doi == 'not found': # if there is no doi in the original reference
      status[i] = 'no DOI'
      if verbose == True:
        print('\033[33m' + str(i+1) +" " +  "DOI missing in reference list" + '\033[0m')
      continue

    try:
        # Search PubMed for the DOI to get the PMID
        # Using a more specific search term to improve accuracy
        search_term = f'{doi}[doi]'
        search_handle = Entrez.esearch(db="pubmed", term=search_term, retmax=10) # Use retmax to limit results
        search_record = Entrez.read(search_handle)
        search_handle.close()

        pubmed_ids = search_record["IdList"]

        if pubmed_ids:
            # If PMID(s) found, fetch the article details to confirm existence and get title
            # Fetch details for the first result, as we expect only one match for a DOI
            fetch_handle = Entrez.efetch(db="pubmed", id=pubmed_ids[0], retmode="xml")
            article_record = Entrez.read(fetch_handle)
            fetch_handle.close()

            if 'PubmedArticle' in article_record and len(article_record['PubmedArticle']) > 0:
                article = article_record['PubmedArticle'][0]['MedlineCitation']['Article']
                title = article.get('ArticleTitle', 'No Title Available')

                if verbose == True:
                   print('\033[32m'  + str(i+1) + " " + title + '\033[0m')
                status[i] = "article found"
            else:
                # This case is less likely if a PMID was found, but included for robustness
                if verbose == True:
                   print('\033[31m' + str(i+1) + " " +  f"Could not fetch details for DOI: {doi}" + '\033[0m')
                status[i] = 'no article'

        else:
            if verbose == True:
               print('\033[31m' + str(i+1) + " " +  f"No article found on PubMed for DOI: {doi}" + '\033[0m')
            status[i] = 'no article' # No PMID found for the DOI

    except Exception as e: # Catch potential errors during Entrez interaction
        if verbose == True:
           print('\033[31m' + str(i+1) + " " +  f"An error occurred checking DOI '{doi}' on PubMed: {e}" + '\033[0m')
        status[i] = 'no article' # Treat any other error as article not found for simplicity

  return status

In [129]:
x = get_references_from_doi(doi =  '10.3389/fmed.2024.1348884', return_dataframe=True, verbose = True)

Sending request...
Processing response for: Efficacy of acupuncture in animal models of various ovarian dysfunctions: a systematic review and meta-analysis
1 Reproductive endocrinology: Restoring ovarian function.
2 Aging of the human ovary and testis.
3 Polycystic ovary syndrome: definition, aetiology, diagnosis and treatment.
4 Expanding our knowledge of premature ovarian insufficiency.
5 The Ovarian Hyperstimulation Syndrome.
6 High concentration of chemerin caused by ovarian hyperandrogenism may lead to poor IVF outcome in polycystic ovary syndrome: a pilot study.
7 Risks and benefits of estrogen plus progestin in healthy postmenopausal women: principal results From the Women's Health Initiative randomized controlled trial.
8 Multi-system reproductive metabolic disorder: significance for the pathogenesis and therapy of polycystic ovary syndrome (PCOS).
9 The Pathogenesis of Polycystic Ovary Syndrome (PCOS): The Hypothesis of PCOS as Functional Ovarian Hyperandrogenism Revisited.
10

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [130]:
doi_list = ['10.1073/pnas.2426992122',"10.1172/JCI173193", "10.1186/s13023-025-03778-1", '10.1186/s13287-025-04610-0', '10.1186/s12951-025-03630-5', '10.1186/s12916-025-04358-7', '10.1186/s12967-025-06991-5']

In [ ]:
for doi in doi_list:
  get_references_from_doi(doi = doi, return_dataframe=True, verbose = False)

Sending request...
Processing response for: Therapeutic restoration of mitochondria–endoplasmic reticulum cross talk for osteoarthritis


/usr/local/lib/python3.12/dist-packages/Bio/Entrez/Parser.py:916: ResourceWarning: unclosed <socket.socket fd=59, family=2, type=1, proto=6, laddr=('172.28.0.12', 40930), raddr=('52.23.19.63', 443)>
  def elementDecl(self, name, model):


Done, found 60 existing documents, 1 non-existing documents and 2 missing DOI
Sending request...
Processing response for: Integrated screening identifies GPR31 as a key driver and druggable target for metabolic dysfunction–associated steatohepatitis
